# Initial Environment setup
- After opening this folder in VS Code:
    - Press ``Ctrl + Shift + ` `` to bring up the terminal
- Run the following command:
    - `conda env create -f environment.yaml`
    - This will have Conda install environment and setup [Google Vizier's Open Source Optimizer](https://github.com/google/vizier)

# Vizier Basics
Below, we provide examples of how to:

*   Define a problem statement and study configuration.
*   Start a client.
*   (Optionally) Connect the client to a server.
*   Perform a typical tuning loop.
*   Use other client APIs.

In [13]:
from vizier import service
from vizier.service import clients
from vizier.service import pyvizier as vz
from vizier.service import servers

## Setting up the problem statement
Here we setup the problem statement, which contains information about the search space and the metrics to optimize.

In [32]:
problem = vz.ProblemStatement()
problem.search_space.root.add_float_param('w', 0.0, 1.0)
problem.search_space.root.add_float_param('x', 0.0, 1.0)
problem.search_space.root.add_float_param('y', 0.0, 1.0)
problem.search_space.root.add_float_param('z', 0.0, 1.0)
problem.metric_information.append(
    vz.MetricInformation(
        name='maximize_metric', goal=vz.ObjectiveMetricGoal.MAXIMIZE))


def evaluate(a: float, b: float, c: float, d: float) -> int:
  return -a * (2*b)**2 - (c - (1*d))**2

## Setting up the study configuration
The study configuration contains additional information, such as the algorithm to use and level of noise that we think the objective will have.

In [33]:
study_config = vz.StudyConfig.from_problem(problem)
study_config.algorithm = 'GP_UCB_PE'

## Setting up the client
Starts a `study_client`, which can be either in **local mode (default)** or **distributed mode.**

**Local Mode:** The client has no `endpoint` set, and will implicitly create a local Vizier Service which will be shared across other clients in the same Python process. Studies will then be stored locally in a SQL database file located at `service.VIZIER_DB_PATH`.

In [34]:
study_client = clients.Study.from_study_config(study_config, owner='owner', study_id='example_study_id')
print('Local SQL database file located at: ', service.VIZIER_DB_PATH)

Local SQL database file located at:  /home/awkwabear/anaconda3/envs/bayop/lib/python3.10/site-packages/vizier/_src/service/vizier.db


**Distributed mode:** The service may be explicitly created, wrapped as a server in a separate process to accept requests from all other client processses. Details such as the `database_url`, `port`, `policy_factory`, etc. can be configured in the server's initializer.

All client processes (on a single machine or over multiple machines) will connect to this server via a globally specified `endpoint`.

In [35]:
# server = servers.DefaultVizierServer()  # Ideally created on a separate process such as a server machine.
# clients.environment_variables.server_endpoint = server.endpoint  # Server address.
# study_client = clients.Study.from_study_config(study_config, owner='owner', study_id = 'example_study_id')  # Now connects to the explicitly created server.

## Client Parallelization
Regardless of whether the setup is local or distributed, we may simultaneously create multiple clients to work on the same study, useful for parallelizing evaluation workload.

In [36]:
another_study_client = clients.Study.from_resource_name(study_client.resource_name)

## Obtaining suggestions
Start requesting suggestions from the server, for evaluating objectives. Suggestions can be made sequentially (`count=1`) or in batches (`count>1`).

In [38]:
print(suggestion.parameters)

{'x': 0.9999999999999991, 'y': 5.975824403371409e-09}


In [41]:
for i in range(10):
  suggestions = study_client.suggest(count=5)
  print(suggestion.parameters)
  # for suggestion in suggestions:
  #   a = suggestion.parameters['a']
  #   b = suggestion.parameters['b']
  #   c = suggestion.parameters['c']
  #   d = suggestion.parameters['d']
  #   objective = evaluate(a, b, c, d)
  #   print(f'Iteration {i}, suggestion ({a},{b},{c},{d}) led to objective value {objective}.')
  #   final_measurement = vz.Measurement({'maximize_metric': objective})
  #   suggestion.complete(final_measurement)

{'x': 0.9999999999999991, 'y': 5.975824403371409e-09}
{'x': 0.9999999999999991, 'y': 5.975824403371409e-09}
{'x': 0.9999999999999991, 'y': 5.975824403371409e-09}
{'x': 0.9999999999999991, 'y': 5.975824403371409e-09}
{'x': 0.9999999999999991, 'y': 5.975824403371409e-09}
{'x': 0.9999999999999991, 'y': 5.975824403371409e-09}
{'x': 0.9999999999999991, 'y': 5.975824403371409e-09}
{'x': 0.9999999999999991, 'y': 5.975824403371409e-09}
{'x': 0.9999999999999991, 'y': 5.975824403371409e-09}
{'x': 0.9999999999999991, 'y': 5.975824403371409e-09}


## Find optimal trial
Find the best objective so far, with corresponding suggestion value. For multiobjective cases, there may be multiple outputs of `optimal_trials()`, all corresponding to a Pareto-optimal curve.

In [ ]:
for optimal_trial in study_client.optimal_trials():
  optimal_trial = optimal_trial.materialize()
  print("Optimal Trial Suggestion and Objective:", optimal_trial.parameters,
        optimal_trial.final_measurement)

## Other client commands
The `study_client` can also send other requests, such as the following:

In [ ]:
study_client.get_trial(1)  # Get the first trial.
study_client.trials()  # Get all trials so far.

# Obtain only the completed trials.
trial_filter = vz.TrialFilter(status=[vz.TrialStatus.COMPLETED])
study_client.trials(trial_filter=trial_filter)